In [1]:
from g4f.client import Client, ClientFactory
import pandas as pd
import numpy as np

/home/alex/Рабочий стол/russian_laws/.venv/lib/python3.12/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
def generate_complaint(client, article):
    prompt = (f"""
        Сгенерируй вопрос, на который можно ответить, используя данную статью. Выбери один из типов вопросов:

        - Прямой: конкретно по тексту статьи (например, "статья 158 жилищного кодекса...")  
        - Семантический: вопрос по смыслу статьи, с применением к реальной ситуации (например, "как оспорить отказ в предоставлении жилья...")  
        - Контекстный: вопрос, требующий понимания правовых связей или ситуации (например, "права собственника при сносе дома...")  

        Статья для анализа:
        {article}
 
        В вопросе не должны фигурировать названия статей и их номеров и подпунктов.
        Не генерируй длинные вопросы.
        В ответе выдай только вопрос, без пояснений.
    """)
    # client = Client()
    response = client.chat.completions.create(
        model="deepseek-v3",
        messages=[{"role": "user", "content": prompt}],
    )

    # response = client.chat.completions.create(
    #     model="turbo",
    #     messages=[{"role": "user", "content": prompt}],
    # )
    return response.choices[0].message.content

In [4]:
client = Client()

generate_complaint(client, "Привет")

KeyboardInterrupt: 

In [ ]:
df = pd.read_csv("../data/processed/articles.csv")
df = df[df.isna().any(axis=1)]

queries = []
i = df.index[0]

# client = ClientFactory.create_client("custom:srv_mjlq1ncq8a3f7fe0aea0", api_key="g4f_api_key")
client = Client()
while i <= df.index[-1]:
    try:
        query = generate_complaint(client, df["Текст статьи"][i][:8000])
        if query:
            queries.append(query)
            i += 1
            print(f"\rИтерация: {i}", end="", flush=True)
    except Exception as e:
        print(f"\rError: {e} in {i}", end="", flush=True)

Итерация: 2159 is required for Puter.js API in 1988

In [192]:
df.to_csv("../data/processed/articles.csv", index=False)